In [1]:
version = "REPLACE_PACKAGE_VERSION"

---
# Assignment 1 Part 2: N-gram Language Models (Cont.) (30 pts)

In this assignment, we're going to train an n-gram language model that is able to "imitate" William Shakespeare's writing. 

In [2]:
# Configure nltk

import nltk

nltk_data_path = "assets/nltk_data"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rein\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Copy and paste the functions you wrote in Part 1 here and import any libraries necessary
# We have tried a more elegant solution by using
# from ipynb.fs.defs.assignment1_part1 import load_data, build_vocab, build_ngrams
# but it doesn't work with the autograder...

def load_data():
    """
    Load text data from a file and produce a list of token lists
    """
    
    from nltk.tokenize import sent_tokenize, TweetTokenizer, word_tokenize
    import re
    
    sentences = []
    
    with open('assets/gutenberg/THE_SONNETS.txt', 'r') as file:
        sentences = file.read()[3:]
        
    # Fix decoding error
    sentences = sentences.encode('cp1252') 
    sentences = sentences.decode('utf-8')
    
    sentences = [i.strip(' ').lower() for i in sentences.split('\n') if len(i.strip(' ').split(' ')) > 3]
    sentences = [nltk.word_tokenize(i) for i in sentences]
    
    return sentences

stu_ans = load_data()

def unique(str_list): # custom helper function for a cleaner build_vocab function
    
    output = []
    
    for x in str_list:
        if x not in output:
            output.append(x)
    output.append('<s>')
    output.append('</s>')
    
    return output

def build_vocab(sentences):
    """
    Take a list of sentences and return a vocab
    """
    
    vocab = []
    sentences_flat = [item for sublist in sentences for item in sublist]
    
    # YOUR CODE HERE
    vocab = unique(sentences_flat)
    
    return vocab

from nltk import ngrams
def build_ngrams(n, sentences):
    """
    Take a list of unpadded sentences and create all n-grams as specified by the argument "n" for each sentence
    """
    
    all_ngrams = []
    
    # YOUR CODE HERE
    for sample in sentences:
        
        tmp = sample.copy()
        
        for i in range(n-1):
            tmp.insert(0, '<s>')
            tmp.append('</s>')
            
        all_ngrams.append([i for i in ngrams(tmp, n)])

    return all_ngrams

## Question 4: Guess the next token (20 pts)

With the help of the three functions you wrote in Part 1, let's first answer the following question as a review on $n$-grams.

Assume we are now working with bi-grams. What is the most likely token that comes after the sequence `<s> <s> <s>`, and how likely? Remember that a bi-gram language model is essentially a first-order Markov Chain. So, what determines the next state in a first-order Markov Chain? 

**Complete the function below to return a `tuple`, where `tuple[0]` is a `str` representing the mostly likely token and `tuple[1]` is a `float` representing its (conditional) probability of being the next token.**

In [5]:
sentences = load_data()
bigrams = build_ngrams(2, sentences)
prev_token = '<s>'
bigram_matches = [[j for j in i if j[0] == prev_token][0] for i in bigrams]
bigram_freq = dict(collections.Counter(bigram_matches))
next_token = max(bigram_freq, key=bigram_freq.get)[1]
prob = bigram_freq[max(bigram_freq, key=bigram_freq.get)] / sum(bigram_freq.values())

NameError: name 'collections' is not defined

In [ ]:
import collections # easier frequency count

def bigram_next_token(start_tokens=("<s>", ) * 3):
    """
    Take some starting tokens and produce the most likely token that follows under a bi-gram model
    """
    
    # load dataset
    sentences = load_data()
    
    # build the bigrams
    bigrams = build_ngrams(2, sentences)
    
    # get the prev_token (Wn_1) which should be the last token in start_tokens
    prev_token = start_tokens[-1]
    
    # get all the bigrams that starts with prev_token (using list comprehension to parse through 2d list of tuples)
    bigram_matches = [[j for j in i if j[0] == prev_token][0] for i in bigrams]
    
    # use collections.Counter to count the occurences of each unique bigram tuple
    bigram_freq = dict(collections.Counter(bigram_matches))
    
    # the most probably next_token is the one that has the highest probability (which also means it has the highest frequency)
    next_token = max(bigram_freq, key=bigram_freq.get)[1]
    
    # calculate the probability of the most occuring bigram by dividing the frequency value of the bigram 
    # to the total population of all bigrams that begins with prev_token
    # i.e. P(W_n | W_n-1) = P(W_n-1, W_n) / P(W_n-1) where W_n-1 = prev_token (or '<s>') and W_n is the word we're looking for.
    prob = bigram_freq[max(bigram_freq, key=bigram_freq.get)] / sum(bigram_freq.values())
    
    return next_token, prob

In [6]:
# Autograder tests

stu_ans = bigram_next_token(start_tokens=("<s>", ) * 3)

assert isinstance(stu_ans, tuple), "Q4: Your function should return a tuple. "
assert len(stu_ans) == 2, "Q4: Your tuple should have two elements. "
assert isinstance(stu_ans[0], str), "Q4: tuple[0] should be a str. "
assert isinstance(stu_ans[1], float), "Q4: tuple[1] should be a float. "

# Some hidden tests


del stu_ans

NameError: name 'bigram_next_token' is not defined

## Question 5: Train an $n$-gram language model (10 pts)

Now we are well positioned to start training an $n$-gram language model. We can fit a language model using the `MLE` class from `nltk.lm`. It requires two inputs: a list of all $n$-grams for each sentence and a vocabulary, both of which you have already written a function to build. Now it's time to put them together to work. 

**Complete the function below to return an `nltk.lm.MLE` object representing a trained $n$-gram language model.**

In [7]:
from nltk.lm import MLE

def train_ngram_lm(n):
    """
    Train a n-gram language model as specified by the argument "n"
    """

    lm = MLE(n)
    
    # YOUR CODE HERE
    
    sentences = load_data()
    vocab = build_vocab(sentences)
    
    for i in range(n, 0, -1):
        lm.fit(build_ngrams(i, sentences), vocab)
    
    return lm

In [8]:
# Autograder tests

stu_n = 4
stu_lm = train_ngram_lm(stu_n)
stu_vocab = build_vocab(load_data())

assert isinstance(stu_lm, nltk.lm.MLE), "Q3b: Your function should return an nltk.lm.MLE object. "

assert hasattr(stu_lm, "vocab") and len(stu_lm.vocab) == len(stu_vocab) + 1, "Q3b: Your language model wasn't trained properly. "

del stu_n, stu_lm, stu_vocab

FINALLY, are you ready to compose sonnets like the real Shakespeare?! We provide some starter code below, but absolutely feel free to modify any parts of it on your own. It'd be interesting to see how the "authenticity" of the sonnets is related to the parameter $n$. Do the sonnets feel more Shakespeare when you increase $n$? 

In [11]:
# Every time it runs, depending on how drunk it is, a different sonnet is written. 
n = 6
num_lines = 14
num_words_per_line = 8
text_seed = ["<s>"] * (n - 1)

lm = train_ngram_lm(n)

sonnet = []
while len(sonnet) < num_lines:
    while True:  # keep generating a line until success
        try:
            line = lm.generate(num_words_per_line, text_seed=text_seed)
        except ValueError:  # the generation is not always successful. need to capture exceptions
            continue
        else:
            line = [x for x in line if x not in ["<s>", "</s>"]]
            sonnet.append(" ".join(line))
            break

# pretty-print your sonnet
print("\n".join(sonnet))

but if thou live remembered not to be
making dead wood more blest than living lips
beggared of blood to blush through lively veins
then bettered that the world may see my
and right perfection wrongfully disgraced ,
and there reigns love and all love ’
or some fierce thing replete with too much
shall hate be fairer lodged than gentle love
whilst my poor lips which should that harvest
to know my shames and praises from your
than that , which on thy humour doth
and your true rights be termed a poet
what need ’ st thou wound with cunning
i summon up remembrance of things past ,
